In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = "notMNIST.pickle"
with open(pickle_file, "rb") as fp:
    all_data = pickle.load(fp)
    train_dataset = all_data["train_dataset"]
    train_labels = all_data["train_labels"]
    validation_dataset = all_data["valid_dataset"]
    validation_labels = all_data["valid_labels"]
    test_dataset = all_data["test_dataset"]
    test_labels = all_data["test_labels"]
    del all_data
    print("Training set: {} {}".format(train_dataset.shape, train_labels.shape))
    print("Validation set: {} {}".format(validation_dataset.shape, validation_labels.shape))
    print("Test set: {} {}".format(test_dataset.shape, test_labels.shape))

Training set: (200000, 28, 28) (200000,)
Validation set: (10000, 28, 28) (10000,)
Test set: (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape(-1, image_size * image_size)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
validation_dataset, validation_labels = reformat(validation_dataset, validation_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print("Training set: {} {}".format(train_dataset.shape, train_labels.shape))
print("Validation set: {} {}".format(validation_dataset.shape, validation_labels.shape))
print("Test set: {} {}".format(test_dataset.shape, test_labels.shape))

Training set: (200000, 784) (200000, 10)
Validation set: (10000, 784) (10000, 10)
Test set: (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    num_hits = np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
    return 100.0 * num_hits / predictions.shape[0]

In [28]:
batch_size = 128
nodes_layer_1 = 1024
nodes_layer_2 = 256
graph = tf.Graph()
beta_regularisation = 1e-3 

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))    
    tf_validation_dataset = tf.constant(validation_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    global_step = tf.Variable(0)

    weights_1 = tf.Variable(tf.truncated_normal(
        [image_size * image_size, nodes_layer_1],
        stddev=np.sqrt(2.0 / (image_size * image_size))))
    weights_2 = tf.Variable(tf.truncated_normal(
        [nodes_layer_1, nodes_layer_2],
        stddev=np.sqrt( 2.0 / nodes_layer_1)))
    weights_3 = tf.Variable(tf.truncated_normal(
        [nodes_layer_2, num_labels],
        stddev=np.sqrt(2.0 / nodes_layer_2)))
    
    bias_1 = tf.Variable(tf.zeros(nodes_layer_1))
    bias_2 = tf.Variable(tf.zeros(nodes_layer_2))
    bias_3 = tf.Variable(tf.zeros(num_labels))
    
    logits_train_1 = tf.matmul(tf_train_dataset, weights_1) + bias_1
    relu_train_1 = tf.nn.relu(logits_train_1)
    dropout_train_1 = tf.nn.dropout(relu_train_1, 0.5)

    logits_train_2 = tf.matmul(dropout_train_1, weights_2) + bias_2
    relu_train_2 = tf.nn.relu(logits_train_2)
    dropout_train_2 = tf.nn.dropout(relu_train_2, 0.5)
    
    logits_train_3 = tf.matmul(dropout_train_2, weights_3) + bias_3
    
    prediction_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
                                                                 logits=logits_train_3))
    
    l2_train_1 = tf.nn.l2_loss(weights_1) * beta_regularisation
    l2_train_2 = tf.nn.l2_loss(weights_2) * beta_regularisation
    l2_train_3 = tf.nn.l2_loss(weights_3) * beta_regularisation
    
    loss = prediction_loss + l2_train_1 + l2_train_2 + l2_train_3
    
    learning_rate = tf.train.exponential_decay(0.5, global_step,
                                              1000, 0.65, staircase=True)
    optimiser = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step)
    
    train_prediction = tf.nn.softmax(logits_train_3)
    
    relu_validation_1 = tf.nn.relu(tf.matmul(tf_validation_dataset, weights_1) + bias_1)
    relu_validation_2 = tf.nn.relu(tf.matmul(relu_validation_1, weights_2) + bias_2)
    validation_prediction = tf.nn.softmax(tf.matmul(relu_validation_2, weights_3) + bias_3)

    relu_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + bias_1)
    relu_test_2 = tf.nn.relu(tf.matmul(relu_test_1, weights_2) + bias_2)
    test_prediction = tf.nn.softmax(tf.matmul(relu_test_2, weights_3) + bias_3)



In [27]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialised")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_dataset = train_dataset[offset:(offset + batch_size), : ]
        batch_labels = train_labels[offset:(offset + batch_size), : ]
        
        feed_dict = {tf_train_dataset: batch_dataset,
                     tf_train_labels: batch_labels}
        
        _, loss_step, prediction_step = session.run([optimiser, loss, train_prediction],
                                                    feed_dict = feed_dict)
        
        if step % 500 == 0:
            print("Minibatch Loss at step {}: {}".format(step, loss_step))
            print("Minibatch Training Accuracy: {}%".format(accuracy(prediction_step, batch_labels)))
            print("Minibatch Validation Accuracy: {}%".format(accuracy(validation_prediction.eval(), validation_labels)))

    print("Test Accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

Initialised
Minibatch Loss at step 0: 3.675792932510376
Minibatch Training Accuracy: 9.375%
Minibatch Validation Accuracy: 25.98%
Minibatch Loss at step 500: 1.1227883100509644
Minibatch Training Accuracy: 87.5%
Minibatch Validation Accuracy: 84.27%
Minibatch Loss at step 1000: 1.0065196752548218
Minibatch Training Accuracy: 85.15625%
Minibatch Validation Accuracy: 85.04%
Minibatch Loss at step 1500: 0.7261525392532349
Minibatch Training Accuracy: 89.84375%
Minibatch Validation Accuracy: 86.29%
Minibatch Loss at step 2000: 0.6387689113616943
Minibatch Training Accuracy: 92.96875%
Minibatch Validation Accuracy: 86.78%
Minibatch Loss at step 2500: 0.7091624736785889
Minibatch Training Accuracy: 86.71875%
Minibatch Validation Accuracy: 87.02%
Minibatch Loss at step 3000: 0.6559233069419861
Minibatch Training Accuracy: 88.28125%
Minibatch Validation Accuracy: 87.26%
Minibatch Loss at step 3500: 0.6370564103126526
Minibatch Training Accuracy: 86.71875%
Minibatch Validation Accuracy: 87.98%
